### Passo 1 - Importar Arquivos e Bibliotecas

In [2]:
# Importando bibliotecas
import pandas as pd
import win32com.client as win32
import pathlib

# Importando bases de dados
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\lojas.csv', encoding= 'latin1', sep=';')
#display(lojas)

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [3]:
# Incluir o nome da loja em vendas

vendas = vendas.merge(lojas, on='ID Loja')
#display(vendas)

In [4]:
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]
#display(dicionario_lojas['Norte Shopping'])
#display(dicionario_lojas['Salvador Shopping'])

In [5]:
dia_indicador = vendas['Data'].max()
print(dia_indicador)
print(f'{dia_indicador.day}/{dia_indicador.month}')

2019-12-26 00:00:00
26/12


### Passo 3 - Salvar a planilha na pasta de backup

In [6]:
# Identificar se a pasta já existe
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]



for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

    # Salvar dentro da pasta    
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)


### Passo 4 - Calcular o indicador para 1 loja

In [7]:
# Definição de metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_prod_dia = 4
meta_prod_ano = 120
meta_ticket_dia = 500
meta_ticket_ano = 500

In [9]:
for loja in dicionario_lojas:

    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']== dia_indicador, :]

    # Faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # Diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()

    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()



### Passo 5 - Enviar por e-mail para o gerente

In [15]:
outlook = win32.Dispatch('outlook.application')

nome = emails.loc[emails['Loja']==loja,'Gerente'].values[0]
mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']==loja,'E-mail'].values[0]
mail.Subject = f'One Page Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
mail.Body = 'Texto do E-mail'

# Anexos
attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print(f'E-mail da Loja {loja} enviado')

com_error: (-2147467260, 'Operação anulada', None, None)

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [16]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by= 'Valor Final', ascending= False)
display(faturamento_lojas_ano)

vendas_dia = vendas.loc[vendas['Data']== dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by= 'Valor Final', ascending= False)
display(faturamento_lojas_dia)

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### Passo 8 - Enviar e-mail para diretoria